In [5]:
!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
import cv2
import scipy.misc
from PIL import Image
from IPython.core.ultratb import AutoFormattedTB
from sklearn.metrics import confusion_matrix
import pandas as pd

__ITB__ = AutoFormattedTB(mode = 'Plain',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

In [2]:
# Mount drive
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
data_path = "/content/drive/My Drive/data/"
img_path = '/tmp/ar_data/'

In [0]:
# Unzip the ar_tag images, and write them to current
zip_ref = zipfile.ZipFile("{}ar_data.zip".format(data_path), 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
# function for showing imgs
def show_img(img_path, name):
    img = cv2.imread('{}{}'.format(img_path, name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [0]:
# helper for combining pos and neg example csv's
def combine_pos_neg_csv(data_path, pos_filename, neg_filename, combined_filename):
    # open csv's
    pos_data = pd.read_csv('{}{}'.format(data_path, pos_filename))
    neg_data = pd.read_csv('{}{}'.format(data_path, neg_filename))
    print(pos_data.head())
    print(neg_data.head())
    # combine csv data
    data = pd.concat([pos_data, neg_data], ignore_index=True)
    # randomly shuffle
    print(data.head())
    data = data.sample(frac=1).reset_index(drop=True)
    data.reset_index(inplace=True)
    data.drop(columns=['index','Unnamed: 0'], inplace=True)
    print(data.head())
    # write to drive
    data.to_csv("{}{}".format(data_path, combined_filename))

In [0]:
# combine_pos_neg_csv(data_path, 'pos_ar_labels.csv', 'neg_ar_labels.csv', 'ar_labels.csv')

In [0]:
def write_train_test_split(data_path, file, frac):
    # open file as csv
    data = pd.read_csv('{}{}'.format(data_path, file))
    data.drop(columns=['Unnamed: 0'], inplace=True)
    # split into train and test sets
    train_set = data.sample(frac=frac, random_state=0)
    train_set.reset_index(inplace=True, drop=True)
    test_set = data.drop(train_set.index)
    test_set.reset_index(inplace=True, drop=True)
    # train_set.drop(columns=['Unnamed: 0'], inplace=True)
    # test_set.drop(columns=['Unnamed: 0'], inplace=True)
    # write sets
    train_set.to_csv('{}train_{}'.format(data_path, file))
    test_set.to_csv('{}test_{}'.format(data_path, file))

In [0]:
# write_train_test_split(data_path, 'ar_labels.csv', 0.75)

In [0]:
# Read in data
data = pd.read_csv('{}ar_labels.csv'.format(data_path))
train_data = pd.read_csv('{}train_ar_labels.csv'.format(data_path))
test_data = pd.read_csv('{}test_ar_labels.csv'.format(data_path))

In [0]:
show_img(img_path, train_data.iloc[1212]['name'])

In [0]:
12